In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import load_matlab_model, write_sbml_model
model= cobra.io.read_sbml_model("Recon3D.xml") #Utilizaremos como base el modelo recon3d, el cual contiene mas reacciones, metabolitos e información de la red metabolica humana (actualmente). 


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
model

Name,Recon3D
Memory address,0x07f67dd5de3a0
Number of metabolites,5835
Number of reactions,10600
Number of groups,0
Objective expression,1.0*BIOMASS_maintenance - 1.0*BIOMASS_maintenance_reverse_5b3f9
Compartments,"cytosol, lysosome, mitochondria, endoplasmic reticulum, extracellular space, peroxisome/glyoxysome, nucleus, golgi apparatus, inner mitochondrial compartment"


In [13]:

#Agregar rx 1 
R1 = Reaction('cit_nt')
R1.name = 'cit_nt'
R1.subsystem = ''
R1.lower_bound = -1000.  # This is the default
R1.upper_bound = 1000.  # This is the default

#Agregar rx 2 
R2 = Reaction('oaa_nt')
R2.name = 'oaa_nt'
R2.subsystem = ''
R2.lower_bound = -1000.  # This is the default
R2.upper_bound = 1000.  # This is the default

#Agregar rx 3 
R3 = Reaction('acl_n')
R3.name = 'ATP-Citrate lyase, nuclear'
R3.subsystem = ''
R3.lower_bound = 0.  # This is the default
R3.upper_bound = 1000.  # This is the default

#Agregar rx 4 PACYL: accoa_n + prot_n --> protac_n + coa_n 
R4 = Reaction('PACYL')
R4.name = 'PACYL'
R4.subsystem = ''
R4.lower_bound = 0.  # This is the default
R4.upper_bound = 1000.  # This is the default

#Agregar rx 5 EX_PAC: protac_n -->  
R5 = Reaction('EX_PAC')
R5.name = 'protein acetylation exchange'
R5.subsystem = 'acetylation exchange'
R5.lower_bound = 0.  # This is the default
R5.upper_bound = 1000.  # This is the default

#Agregar rx 6 EX_P    --> prot_n
R6 = Reaction('EX_P')
R6.name = 'nuclear protein consume reaction'
R6.subsystem = 'nuclear protein'
R6.lower_bound = 0.  # This is the default
R6.upper_bound = 1000.  # This is the default



In [14]:
#Metabolites

cit_n= Metabolite(
    'cit_n',
    formula='C6H5O7',
    name='Citrate',
    compartment='n')

cit_c= Metabolite(
    'cit_c',
    formula='C6H5O7',
    name='Citrate',
    compartment='c')

oaa_n= Metabolite(
    'oaa_n',
    formula='C4H2O5',
    name='Oxaloacetate',
    compartment='n')

coa_n= Metabolite(
    'coa_n',
    formula='C21H32N7O16P3S',
    name='Coenzyme A',
    compartment='n')

accoa_n= Metabolite(
    'accoa_n',
    formula='C23H34N7O17P3S',
    name='Acetyl-CoA',
    compartment='n')

accoa_c= Metabolite(
    'accoa_c',
    formula='C23H34N7O17P3S',
    name='Acetyl-CoA',
    compartment='c')

coa_c= Metabolite(
    'coa_c',
    formula='C21H32N7O16P3S',
    name='Coenzyme A',
    compartment='c')

oaa_c= Metabolite(
    'oaa_c',
    formula='C4H2O5',
    name='Oxaloacetate',
    compartment='c')

atp_n= Metabolite(
    'atp_n',
    formula='C10H12N5O13P3',
    name='atp',
    compartment='n')

adp_n= Metabolite(
    'adp_n',
    formula='C10H12N5O10P2',
    name='adp',
    compartment='n')

pi_n= Metabolite(
    'pi_n',
    formula='HO4P',
    name='Phosphate',
    compartment='n')

prot_n= Metabolite(
    'prot_n',
    formula='',
    name='Protein nucleus',
    compartment='n')

protac_n= Metabolite(
    'protac_n',
    formula='',
    name='Protein acetilation nucleus',
    compartment='n')

protac_c= Metabolite(
    'protac_c',
    formula='',
    name='Protein acetilation citosol',
    compartment='c')

prot_e= Metabolite(
    'prot_e',
    formula='',
    name='Protein extracellular',
    compartment='e')

prot_c= Metabolite(
    'prot_c',
    formula='',
    name='Protein citosol',
    compartment='c')





In [15]:
#Agregar metabolitos a la rx.

R1.add_metabolites({ 
    cit_n: -1.0, 
    cit_c: 1.0, 
})
#R1.reaction 

R2.add_metabolites({ 
    oaa_n: -1.0, 
    oaa_c: 1.0, 
})
#R2.reaction 

R3.add_metabolites({ 
    atp_n: -1.0, 
    cit_n: -1.0, 
    coa_n: -1.0,
    accoa_n: 1.0,
    adp_n: 1.0,
    oaa_n: 1.0,
    pi_n: 1.0,
})
#R3.reaction 

#R4.reaction  accoa_n + prot_n --> protac_n + coa_n
R4.add_metabolites({ 
    accoa_n: -1.0, 
    prot_n: -1.0, 
    protac_n: 1.0,
    coa_n: 1.0,
})


#EX_PAC: protac_n -->  
R5.add_metabolites({ 
    protac_n: -1.0, 

})

#    --> prot_n
R6.add_metabolites({ 
    prot_n: 1.0, 

})


In [16]:
#Verificar rx. agregadas
print(R1)
print(R2)
print(R3)
print(R4)
print(R5)
print(R6)


cit_nt: cit_n <=> cit_c
oaa_nt: oaa_n <=> oaa_c
acl_n: atp_n + cit_n + coa_n --> accoa_n + adp_n + oaa_n + pi_n
PACYL: accoa_n + prot_n --> coa_n + protac_n
EX_PAC: protac_n --> 
EX_P:  --> prot_n


In [17]:
#Incluir gene reaction rule, en este caso solo contamos con la grr en R3
R1.gene_reaction_rule =''
R2.gene_reaction_rule =''
R3.gene_reaction_rule ='(47_AT1 or 47_AT2)'
R4.gene_reaction_rule =''
R5.gene_reaction_rule =''
R6.gene_reaction_rule =''



In [18]:
print(f'{len(model.reactions)} reactions initially')
print(f'{len(model.metabolites)} metabolites initially')
print(f'{len(model.genes)} genes initially')

10600 reactions initially
5835 metabolites initially
2248 genes initially


In [19]:
model.add_reactions([R1,R2,R3,R4,R5,R6])

# The objects have been added to the model
print(f'{len(model.reactions)} reactions')
print(f'{len(model.metabolites)} metabolites')
print(f'{len(model.genes)} genes')

10606 reactions
5839 metabolites
2248 genes


In [20]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
24_25DHVITD3tm : 2425dhvitd3_m --> 2425dhvitd3_c
25HVITD3t : 25hvitd3_c --> 25hvitd3_e
COAtl : coa_c <=> coa_l
EX_5adtststerone_e : 5adtststerone_e <=> 
EX_5adtststerones_e : 5adtststerones_e <=> 
EX_5fthf_e : 5fthf_e <=> 
EX_5htrp_e : 5htrp_e <=> 
EX_5mthf_e : 5mthf_e <=> 
EX_5thf_e : 5thf_e <=> 
EX_6dhf_e : 6dhf_e <=> 
24_25VITD3Hm : 25hvitd3_m + h_m + nadph_m + o2_m --> 2425dhvitd3_m + h2o_m + nadp_m
24NPHte : 24nph_e <=> 24nph_c
10FTHF7GLUtl : 10fthf7glu_c --> 10fthf7glu_l
10FTHFtm : 10fthf_c <=> 10fthf_m
11DOCRTSLtr : 11docrtsl_c <=> 11docrtsl_r
13DAMPPOX : 13dampp_c + h2o_c + o2_c --> bamppald_c + h2o2_c + nh4_c
24_25DHVITD2t : 2425dhvitd2_c --> 2425dhvitd2_e
24_25DHVITD2tm : 2425dhvitd2_m --> 2425dhvitd2_c
24_25DHVITD3t : 2425dhvitd3_c --> 2425dhvitd3_e
25VITD2Hm : 25hvitd2_m + h_m + nadph_m + o2_m --> 1a25dhvitd2_m + h2o_m + nadp_m
2AMACHYD : 2amac_c + h2o_c --> nh4_c + pyr_c
2AMACSULT : 2amac_c + nadph_c + paps_c --> Lcyst_c + nadp_c + pap_c
2AMADPTm : L2aa

In [22]:
#Guardar nuevo modelo con las 6rx necesarias para promover acetilación
cobra.io.write_sbml_model(model, "recon3d_6rx.xml")